In [26]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet18
from tqdm import tqdm
import os
import random
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from collections import OrderedDict

In [3]:
class TwoCropsTransform:
    def __init__(self, t1, t2):
        self.t1 = t1
        self.t2 = t2
    def __call__(self, x):
        return self.t1(x), self.t2(x)

class ConsecutivePairDataset(Dataset):
    def __init__(self, base_dataset):
        self.base = base_dataset
    def __len__(self):
        return len(self.base) - 1
    def __getitem__(self, idx):
        a, _ = self.base[idx]
        b, _ = self.base[idx + 1]
        return a, b

def off_diagonal(x: torch.Tensor) -> torch.Tensor:
    n = x.shape[0]
    assert x.dim() == 2 and x.shape[0] == x.shape[1], "off_diagonal requires square matrix"
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()

In [4]:

class VICRegLoss(nn.Module):
    def __init__(self, sim_coeff=25.0, std_coeff=25.0, cov_coeff=1.0, eps=1e-4):
        super().__init__()
        self.sim_coeff = sim_coeff
        self.std_coeff = std_coeff
        self.cov_coeff = cov_coeff
        self.eps = eps

    def forward(self, z1, z2):
        repr_loss = F.mse_loss(z1, z2)
        z1 = z1 - z1.mean(dim=0)
        z2 = z2 - z2.mean(dim=0)
        std_z1 = torch.sqrt(z1.var(dim=0, unbiased=False) + self.eps)
        std_z2 = torch.sqrt(z2.var(dim=0, unbiased=False) + self.eps)
        std_loss = torch.mean(F.relu(1.0 - std_z1)) + torch.mean(F.relu(1.0 - std_z2))
        B = z1.size(0)
        cov_z1 = (z1.T @ z1) / (B - 1)
        cov_z2 = (z2.T @ z2) / (B - 1)
        cov_loss = (off_diagonal(cov_z1).pow(2).sum() / z1.size(1)) + \
                   (off_diagonal(cov_z2).pow(2).sum() / z2.size(1))
        loss = self.sim_coeff * repr_loss + self.std_coeff * std_loss + self.cov_coeff * cov_loss

        stats = {
            "repr": repr_loss.item(),
            "std": std_loss.item(),
            "cov": cov_loss.item(),
            "total": loss.item()
        }
        return loss, stats

In [5]:
class Projector(nn.Module):
    def __init__(self, in_dim=512, hidden_dim=2048, out_dim=512):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, out_dim)
        )
    def forward(self, x):
        return self.net(x)

class Predictor(nn.Module):
    def __init__(self, in_dim=512, hidden_dim=512, out_dim=512):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, out_dim)
        )
    def forward(self, x):
        return self.net(x)

In [6]:
def get_encoder(feature_dim=512, pretrained=False):
    m = resnet18(pretrained=pretrained)
    m.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # changed from 3→1 channel
    m.fc = nn.Identity()
    return m

def get_transforms(size=28):
    aug = T.Compose([
        T.RandomResizedCrop(size=size, scale=(0.8, 1.0)),
        T.RandomHorizontalFlip(),
        T.RandomApply([T.ColorJitter(0.2, 0.2, 0.2, 0.1)], p=0.5),
        T.RandomGrayscale(p=0.1),
        T.ToTensor(),
        T.Normalize((0.5,), (0.5,))
    ])
    return aug, aug

In [7]:
def train_snd_vic(args):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Device: {device}")

    torch.manual_seed(args.seed)
    np.random.seed(args.seed)
    random.seed(args.seed)
    if device == 'cuda':
        torch.cuda.manual_seed_all(args.seed)

    t1, t2 = get_transforms(size=args.size)

    base_trainset = torchvision.datasets.FashionMNIST(root=args.data_root, train=True, download=True, transform=T.ToTensor())

    if args.use_consecutive:
        base_raw = torchvision.datasets.FashionMNIST(root=args.data_root, train=True, download=True, transform=None)
        pair_dataset = ConsecutivePairDataset(base_raw)

        class AugmentedPairDataset(Dataset):
            def __init__(self, pair_ds, t1, t2):
                self.pair = pair_ds
                self.t1 = t1
                self.t2 = t2
            def __len__(self):
                return len(self.pair)
            def __getitem__(self, idx):
                a, b = self.pair[idx]
                return (self.t1(a), self.t2(b))
        dataset = AugmentedPairDataset(pair_dataset, t1, t2)
    else:
        aug_dataset = torchvision.datasets.FashionMNIST(
            root=args.data_root, train=True, download=True,
            transform=TwoCropsTransform(t1, t2)
        )
        dataset = aug_dataset

    loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=not args.use_consecutive,
                        num_workers=args.num_workers, drop_last=True)

    encoder = get_encoder(pretrained=args.pretrained).to(device)
    projector = Projector(in_dim=512, hidden_dim=args.hidden_dim, out_dim=args.out_dim).to(device)
    predictor = Predictor(in_dim=args.out_dim, hidden_dim=args.pred_hidden, out_dim=args.out_dim).to(device)

    vicreg_loss = VICRegLoss(sim_coeff=args.sim_coeff, std_coeff=args.std_coeff, cov_coeff=args.cov_coeff)

    target_params = list(encoder.parameters()) + list(projector.parameters())
    optimizer_target = torch.optim.Adam(target_params, lr=args.lr, weight_decay=args.wd)
    optimizer_pred = torch.optim.Adam(predictor.parameters(), lr=args.pred_lr, weight_decay=args.pred_wd)

    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_target, T_max=args.epochs)
    os.makedirs(args.save_dir, exist_ok=True)

    for epoch in range(args.epochs):
        encoder.train(); projector.train(); predictor.train()
        running_vic, running_distil = 0.0, 0.0
        pbar = tqdm(loader, desc=f"Epoch {epoch+1}/{args.epochs}", leave=False)
        for batch_idx, batch in enumerate(pbar):
            if args.use_consecutive:
                x1, x2 = batch
            else:
                (x1, x2), _ = batch
            x1, x2 = x1.to(device), x2.to(device)
            h1, h2 = encoder(x1), encoder(x2)
            z1, z2 = projector(h1), projector(h2)

            with torch.no_grad():
                z1_det, z2_det = z1.detach(), z2.detach()
            p1, p2 = predictor(z1_det), predictor(z2_det)

            distil_loss = F.mse_loss(p1, z1_det) + F.mse_loss(p2, z2_det)
            optimizer_pred.zero_grad()
            distil_loss.backward()
            optimizer_pred.step()

            loss_vic, stats_vic = vicreg_loss(z1, z2)
            optimizer_target.zero_grad()
            loss_vic.backward()
            optimizer_target.step()

            running_vic += stats_vic["total"]
            running_distil += distil_loss.item()

            pbar.set_postfix({"vic": f"{running_vic/(batch_idx+1):.4f}", "distil": f"{running_distil/(batch_idx+1):.4f}"})

        scheduler.step()
        if (epoch + 1) % args.save_every == 0 or epoch == args.epochs - 1:
            ckpt = {
                'encoder': encoder.state_dict(),
                'projector': projector.state_dict(),
                'predictor': predictor.state_dict(),
                'epoch': epoch
            }
            torch.save(ckpt, os.path.join(args.save_dir, f"snd_vic_epoch{epoch+1}.pth"))
            print(f"Saved checkpoint epoch {epoch+1}")

        print(f"Epoch {epoch+1:03d} | VIC {running_vic/len(loader):.4f} | DIST {running_distil/len(loader):.4f}")

    print("Training finished on FashionMNIST.")

def parse_args(args=None):
    parser = argparse.ArgumentParser(description="SND-VIC (FashionMNIST example)")
    parser.add_argument('--epochs', type=int, default=100)
    parser.add_argument('--batch-size', type=int, default=256)
    parser.add_argument('--lr', type=float, default=1e-3)
    parser.add_argument('--wd', type=float, default=1e-6)
    parser.add_argument('--hidden-dim', type=int, default=2048)
    parser.add_argument('--out-dim', type=int, default=512)
    parser.add_argument('--pred-hidden', type=int, default=512)
    parser.add_argument('--pred-lr', type=float, default=1e-3)
    parser.add_argument('--pred-wd', type=float, default=1e-6)
    parser.add_argument('--sim-coeff', type=float, default=25.0)
    parser.add_argument('--std-coeff', type=float, default=25.0)
    parser.add_argument('--cov-coeff', type=float, default=1.0)
    parser.add_argument('--save-every', type=int, default=50)
    parser.add_argument('--save-dir', type=str, default='./checkpoints_fmnist')
    parser.add_argument('--num-workers', type=int, default=4)
    parser.add_argument('--data-root', type=str, default='./data')
    parser.add_argument('--size', type=int, default=28)
    parser.add_argument('--pretrained', action='store_true')
    parser.add_argument('--use-consecutive', action='store_true')
    parser.add_argument('--seed', type=int, default=42)
    return parser.parse_args(args)

In [8]:
args = parse_args([
    '--epochs', '100',
    '--batch-size', '128',
    '--lr', '1e-3',
    '--wd', '1e-6',
    '--hidden-dim', '2048',
    '--out-dim', '512',
    '--pred-hidden', '512',
    '--pred-lr', '1e-3',
    '--sim-coeff', '25.0',
    '--std-coeff', '25.0',
    '--cov-coeff', '1.0',
    '--save-every', '50',
    '--num-workers', '0'
])

train_snd_vic(args)

Device: cuda


c:\Users\ASUS\anaconda3\envs\torch_gpu\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\anaconda3\envs\torch_gpu\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 001 | VIC 25.9375 | DIST 0.1005


Epoch 002 | VIC 18.9964 | DIST 0.0673


Epoch 003 | VIC 16.5539 | DIST 0.0841


...

Epoch 096 | VIC 8.6634 | DIST 0.0103


Epoch 097 | VIC 8.6661 | DIST 0.0102


Epoch 098 | VIC 8.6602 | DIST 0.0102


Epoch 099 | VIC 8.6652 | DIST 0.0103


Saved checkpoint epoch 100
Epoch 100 | VIC 8.6568 | DIST 0.0103
Training finished on FashionMNIST.


In [28]:
def get_encoder(feature_dim=512, pretrained=False):
    m = resnet18(weights=None if not pretrained else "IMAGENET1K_V1")

    m.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    m.fc = nn.Identity()
    return m

In [29]:
ckpt_path = "./checkpoints_fmnist/snd_vic_epoch100.pth"
device = "cuda" if torch.cuda.is_available() else "cpu"
k = 5 

encoder = get_encoder(pretrained=False).to(device)
ckpt = torch.load(ckpt_path, map_location=device)
encoder.load_state_dict(ckpt['encoder'])
encoder.eval()
print(f"Načítaný checkpoint z epochy {ckpt['epoch']+1}")

transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.FashionMNIST(root=args.data_root, train=True, download=True, transform=transform)
testset = torchvision.datasets.FashionMNIST(root=args.data_root, train=False, download=True, transform=transform)

train_loader = DataLoader(trainset, batch_size=256, shuffle=False, num_workers=2)
test_loader = DataLoader(testset, batch_size=256, shuffle=False, num_workers=2)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_6544\735516528.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=device)


Načítaný checkpoint z epochy 100


In [30]:
def extract_features(loader, model):
    features, labels = [], []
    with torch.no_grad():
        for x, y in tqdm(loader, desc="Extracting features"):
            x = x.to(device)
            h = model(x)
            h = h.view(h.size(0), -1)
            features.append(h.cpu())
            labels.append(y)
    return torch.cat(features).numpy(), torch.cat(labels).numpy()

train_feats, train_labels = extract_features(train_loader, encoder)
test_feats, test_labels = extract_features(test_loader, encoder)

print(f"Train feats: {train_feats.shape}, Test feats: {test_feats.shape}")

knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
knn.fit(train_feats, train_labels)

preds = knn.predict(test_feats)
acc = accuracy_score(test_labels, preds)
print(f"\nk-NN presnosť (k={k}): {acc*100:.2f}%")

Extracting features: 100%|██████████| 40/40 [00:09<00:00,  4.20it/s]


Train feats: (60000, 512), Test feats: (10000, 512)

k-NN presnosť (k=5): 81.45%
